In [36]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
import pandas as pd

In [38]:
df_star = pd.read_excel('/content/drive/My Drive/PBA/Danantara_Preprocessed.xlsx')

In [39]:
!pip install spacy nlpaug
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 29.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [40]:
!pip install transformers torch

## Mendefinisikan fungsi augmentasi

In [41]:
import spacy
import random


# --- 1. Muat Model spaCy (tetap diperlukan untuk NER) ---
try:
    nlp = spacy.load("en_core_web_sm")
    print("Model 'en_core_web_sm' berhasil dimuat.")
except OSError:
    print("Model 'en_core_web_sm' tidak ditemukan. Mengunduh...")
    spacy.cli.download("en_core_web_sm")
    nlp = spacy.load("en_core_web_sm")
    print("Model 'en_core_web_sm' berhasil diunduh dan dimuat.")

# --- 2. Definisikan Kamus Sinonim Bahasa Indonesia Kustom ---
# Ini adalah contoh. Kamu perlu memperluasnya dengan data sinonim yang lebih banyak.
indonesian_synonyms_dict = {
    # --- Kata Kerja Umum ---
    "sedang": ["tengah", "lagi", "hendak", "sementara"],
    "melakukan": ["mengerjakan", "menjalankan", "melakoni", "mengadakan", "membuat"],
    "mengunjungi": ["mendatangi", "menengok", "bertandang", "singgah", "datang ke"],
    "meresmikan": ["membuka", "meluncurkan", "mengesahkan", "mencanangkan"],
    "mengumumkan": ["menyatakan", "memberitahukan", "memublikasikan", "mewartakan", "mempublikasikan"],
    "mengubah": ["mengganti", "merombak", "memodifikasi", "mengalihkan", "merombak"],
    "menguasai": ["mengendalikan", "memimpin", "mendominasi", "memiliki"],
    "meningkat": ["naik", "bertambah", "melonjak", "bertumbuh", "membesar"],
    "menurun": ["turun", "berkurang", "merosot", "melemah"],
    "memulai": ["mengawali", "memelopori", "merintis", "mencetuskan"],
    "mengakhiri": ["menyudahi", "menuntaskan", "menyelesaikan", "menutup"],
    "memberikan": ["menyediakan", "menyumbangkan", "mewariskan", "menawarkan", "memberi"],
    "mendapatkan": ["memperoleh", "meraih", "menggapai", "memperoleh"],
    "menjelaskan": ["memaparkan", "menerangkan", "menguraikan", "membeberkan"],
    "menyampaikan": ["mengutarakan", "mengemukakan", "menuturkan", "melaporkan"],
    "mengatakan": ["menyebutkan", "menuturkan", "berujar", "menjelaskan"],
    "menemukan": ["mendapati", "menyingkap", "menguak"],
    "mencapai": ["meraih", "menggapai", "menjangkau", "memenuhi"],
    "berhasil": ["sukses", "berjaya", "tercapai"],
    "gagal": ["tidak_berhasil", "mandek", "bangkrut"],
    "bekerja": ["berkarya", "bertugas", "berprofesi"],
    "membantu": ["menolong", "mendukung", "menyokong"],
    "memimpin": ["menakhodai", "mengarahkan", "mengepalai", "mengepalai"],
    "berada": ["terletak", "berlokasi", "bertempat"],
    "terjadi": ["berlangsung", "berlaku", "muncul"],
    "diharapkan": ["diprediksi", "diramalkan", "diproyeksikan", "diagendakan"],
    "mengalami": ["merasakan", "menjalani", "menjumpai"],

    # --- Kata Benda Umum ---
    "perusahaan": ["firma", "korporasi", "badan_usaha", "bisnis", "usaha"],
    "pemerintah": ["negara", "otoritas", "kekuasaan", "rezim"],
    "proyek": ["program", "rencana", "inisiatif", "prakarsa", "agenda"],
    "infrastruktur": ["sarana", "prasarana", "fasilitas"],
    "inovasi": ["penemuan", "terobosan", "kreativitas", "gagasan_baru"],
    "teknologi": ["ilmu_terapan", "sains_terapan", "rekayasa"],
    "kecerdasan": ["intelegensi", "daya_pikir", "akal"],
    "dunia": ["jagad", "bumi", "alam", "semesta"],
    "kota": ["kawasan_perkotaan", "pusat_kota", "metropolis"],
    "jembatan": ["penghubung", "lintasan", "jambatan"], # Jambatan adalah bentuk lama/dialek
    "masyarakat": ["publik", "warga", "komunitas", "khalayak"],
    "investor": ["pemodal", "penanam_modal", "kapitalis"],
    "ekonom": ["ahli_ekonomi", "pakar_ekonomi"],
    "sentimen": ["perasaan", "suasana_hati", "pandangan", "persepsi"],
    "kekhawatiran": ["kecemasan", "keresahan", "kegelisahan"],
    "kinerja": ["prestasi", "unjuk_kerja", "hasil"],
    "lembaga": ["institusi", "badan", "organisasi"],
    "risiko": ["bahaya", "ancaman", "kemungkinan_buruk"],
    "aset": ["aktiva", "harta", "milik"],
    "struktur": ["susunan", "tatanan", "komposisi"],
    "figur": ["tokoh", "sosok", "pribadi"],
    "masalah": ["persoalan", "problem", "kendala"],
    "kunci": ["penentu", "faktor_utama", "dasar"],
    "kesuksesan": ["keberhasilan", "kejayaan", "kemenangan"],
    "integrasi": ["penyatuan", "penggabungan", "kesatuan"],
    "pendapat": ["opini", "pandangan", "gagasan"],
    "analisis": ["kajian", "penelitian", "telaah"],
    "data": ["informasi", "keterangan", "fakta"],
    "sistem": ["mekanisme", "prosedur", "tata_cara"],
    "situasi": ["keadaan", "kondisi", "posisi"],
    "kebijakan": ["aturan", "strategi", "prinsip"],
    "tantangan": ["kendala", "rintangan", "cobaan"],

    # --- Kata Sifat Umum ---
    "penting": ["esensial", "utama", "vital", "berarti", "signifikan"],
    "baru": ["anyar", "terkini", "modern", "mutakhir", "kontemporer"],
    "besar": ["agung", "raya", "kolosal", "raksasa", "luas"],
    "kecil": ["mungil", "mini", "sedikit", "ringan"],
    "kuat": ["kokoh", "tangguh", "perkasa", "solid"],
    "lemah": ["rapuh", "rentan", "ringkih", "loyo"],
    "baik": ["bagus", "positif", "unggul", "bermutu"],
    "buruk": ["jelek", "negatif", "rendah", "bobrok"],
    "cepat": ["laju", "pesat", "deras", "segera"],
    "lambat": ["pelan", "perlahan", "terhambat"],
    "sulit": ["sukar", "berat", "pelik", "rumit"],
    "mudah": ["gampang", "praktis", "sederhana"],
    "akurat": ["tepat", "benar", "cermat", "teliti"],
    "spesifik": ["khusus", "tertentu", "detail"],
    "global": ["dunia", "internasional", "universal"],
    "nasional": ["kebangsaan", "domestik"],
    "positif": ["optimis", "baik", "menguntungkan"],
    "negatif": ["pesimis", "buruk", "merugikan"],
    "eksperimental": ["percobaan", "uji_coba", "rancangan"],

    # --- Kata Keterangan Umum ---
    "saat_ini": ["sekarang", "kini", "dewasa_ini", "belakangan_ini"],
    "ke_depan": ["selanjutnya", "mendatang", "nanti"],
    "sangat": ["amat", "sekali", "sungguh", "betul-betul"],
    "juga": ["pula", "pun", "serta"],
    "sehingga": ["maka", "jadi", "dengan_demikian"],
    "terutama": ["khususnya", "utamanya", "paling_tidak"],
    "hanya": ["cuma", "sekadar", "melulu"],
    "tidak": ["bukan", "tak"],
    "bahkan": ["malah", "lagi_pula"],
    "kemudian": ["lalu", "selanjutnya", "setelah_itu"],
    "sebelum": ["pra"],
    "setelah": ["pasca"],
    "agak": ["sedikit", "lumayan"],
    "cukup": ["memadai", "memadai_cukup"], # Hati-hati dengan ini
    "misalnya": ["contohnya", "umpamanya", "seperti"],
    "demikian": ["begitu", "seperti_itu"],
    "tentu": ["pasti", "jelas", "yakini"],
    "benar": ["sungguh", "betul", "persis"],
    "jelas": ["gamblang", "terang", "lugas"],
    "umumnya": ["biasanya", "lazimnya", "kebanyakan"],
    "khususnya": ["terutama", "istimewanya"],
    "secara": ["berdasarkan", "dengan_cara"],
    "segera": ["lekas", "cepat", "buru-buru"],
    "mendadak": ["tiba-tiba", "sekonyong-konyong"],
    "perlahan": ["pelan-pelan", "bertahap"],
    "lebih": ["terlebih"], # Hati-hati dengan ini
    "kurang": ["minus"], # Hati-hati dengan ini
}
print("Kamus sinonim bahasa Indonesia kustom berhasil didefinisikan.")

# --- 3. Fungsi Augmentasi yang Diperbarui dengan Kamus Kustom ---
def augment_text_with_custom_synonyms(text, nlp_model, custom_synonyms_dict, synonym_rate=0.5, random_seed=None):
    if random_seed is not None:
        random.seed(random_seed)

    doc = nlp_model(text)
    protected_spans = sorted([(ent.start_char, ent.end_char) for ent in doc.ents])

    augmented_parts = []
    last_idx = 0

    for start_ent, end_ent in protected_spans:
        if start_ent > last_idx:
            non_entity_segment = text[last_idx:start_ent].strip()
            if non_entity_segment:
                words = non_entity_segment.split()
                augmented_words = []
                for word in words:
                    # Coba temukan sinonim dari kamus kustom
                    if random.random() < synonym_rate and word.lower() in custom_synonyms_dict:
                        synonyms = custom_synonyms_dict[word.lower()]
                        if synonyms:
                            # Pilih sinonim secara acak dari daftar
                            augmented_words.append(random.choice(synonyms))
                        else:
                            augmented_words.append(word) # Jika tidak ada sinonim, gunakan kata asli
                    else:
                        augmented_words.append(word)
                augmented_parts.append(" ".join(augmented_words))

        augmented_parts.append(text[start_ent:end_ent]) # Tambahkan entitas yang dilindungi
        last_idx = end_ent

    if last_idx < len(text):
        non_entity_segment = text[last_idx:].strip()
        if non_entity_segment:
            words = non_entity_segment.split()
            augmented_words = []
            for word in words:
                if random.random() < synonym_rate and word.lower() in custom_synonyms_dict:
                    synonyms = custom_synonyms_dict[word.lower()]
                    if synonyms:
                        augmented_words.append(random.choice(synonyms))
                    else:
                        augmented_words.append(word)
                else:
                    augmented_words.append(word)
            augmented_parts.append(" ".join(augmented_words))

    return " ".join(augmented_parts).strip().replace("  ", " ")

print("Fungsi 'augment_text_with_custom_synonyms' berhasil didefinisikan.")

# --- 4. Contoh Penggunaan dengan Kamus Kustom ---
# Contoh berita asli (sama seperti sebelumnya)
berita_asli_1 = "BPI Danantara sedang melakukan negosiasi penting dengan pemerintah terkait proyek infrastruktur baru di Jawa Barat."
berita_asli_2 = "Perusahaan teknologi raksasa Google mengumumkan inovasi baru dalam kecerdasan buatan, yang akan mengubah cara kita berinteraksi dengan dunia."
berita_asli_3 = "Presiden Joko Widodo mengunjungi kota Surabaya untuk meresmikan jembatan Suramadu yang baru direnovasi."

print("\n--- Berita Setelah Augmentasi (Kamus Kustom & Melindungi Entitas) ---")

augmented_text_1_custom = augment_text_with_custom_synonyms(
    berita_asli_1, nlp, indonesian_synonyms_dict, synonym_rate=0.4, random_seed=42
)
print(f"Augmented (1) Custom: {augmented_text_1_custom}")

augmented_text_2_custom = augment_text_with_custom_synonyms(
    berita_asli_2, nlp, indonesian_synonyms_dict, synonym_rate=0.4, random_seed=42
)
print(f"Augmented (2) Custom: {augmented_text_2_custom}")

augmented_text_3_custom = augment_text_with_custom_synonyms(
    berita_asli_3, nlp, indonesian_synonyms_dict, synonym_rate=0.4, random_seed=42
)
print(f"Augmented (3) Custom: {augmented_text_3_custom}")

print("\n--- Analisis Entitas pada Teks Asli dan Augmented (Kustom) ---")
for i, (text_orig, text_aug) in enumerate(zip([berita_asli_1, berita_asli_2, berita_asli_3],
                                             [augmented_text_1_custom, augmented_text_2_custom, augmented_text_3_custom])):
    print(f"\n--- Berita {i+1} ---")
    print(f"Original: {text_orig}")
    doc_orig = nlp(text_orig)
    print("  Entities Original:")
    if doc_orig.ents:
        for ent in doc_orig.ents:
            print(f"    - '{ent.text}' (Label: {ent.label_})")
    else:
        print("    Tidak ada entitas terdeteksi.")


    print(f"Augmented: {text_aug}")
    doc_aug = nlp(text_aug)
    print("  Entities Augmented:")
    if doc_aug.ents:
        for ent in doc_aug.ents:
            print(f"    - '{ent.text}' (Label: {ent.label_})")
    else:
        print("    Tidak ada entitas terdeteksi.")

Model 'en_core_web_sm' berhasil dimuat.
Kamus sinonim bahasa Indonesia kustom berhasil didefinisikan.
Fungsi 'augment_text_with_custom_synonyms' berhasil didefinisikan.

--- Berita Setelah Augmentasi (Kamus Kustom & Melindungi Entitas) ---
Augmented (1) Custom: BPI Danantara sedang melakoni negosiasi esensial dengan pemerintah terkait proyek sarana kontemporer di Jawa Barat .
Augmented (2) Custom: Perusahaan sains_terapan raksasa Google mempublikasikan gagasan_baru anyar dalam kecerdasan buatan , yang akan merombak cara kita berinteraksi dengan dunia.
Augmented (3) Custom: Presiden Joko Widodo mengunjungi kota Surabaya untuk meluncurkan jembatan Suramadu yang kontemporer direnovasi.

--- Analisis Entitas pada Teks Asli dan Augmented (Kustom) ---

--- Berita 1 ---
Original: BPI Danantara sedang melakukan negosiasi penting dengan pemerintah terkait proyek infrastruktur baru di Jawa Barat.
  Entities Original:
    - 'BPI Danantara' (Label: ORG)
    - 'Jawa Barat' (Label: PERSON)
Augmented

# df positif menjadi 50

In [42]:
df_positif = df_star[df_star['label']=='positif']

In [43]:
df_positif = df_positif.sample(n=100, random_state=42)

# augmentasi 65 berita negatif


In [44]:
df_negatif = df_star[df_star['label']=='Negatif']

In [45]:
df_negatif_aug = df_negatif.sample(n=65, random_state=42, replace=True)

In [46]:
df_negatif_aug['augmented_content'] = df_negatif_aug['content'].apply(lambda x: augment_text_with_custom_synonyms(
    x, nlp, indonesian_synonyms_dict, synonym_rate=0.4, random_seed=42
))

In [47]:
df_negatif_aug[['content', 'augmented_content']]

,content,augmented_content
135,--\n\nCentre for Strategic and International S...,-- Centre for Strategic and International Stud...
29,Indonesia has launched a new sovereign wealth ...,Indonesia has launched a new sovereign wealth ...
17,KOMPAS.com - Kerugian besar dari skandal korup...,KOMPAS.com - Kerugian agung dari skandal korup...
44,"JAKARTA, KOMPAS — Badan Pengelola Investasi Da...","JAKARTA , KOMPAS — Badan Pengelola Investasi D..."
42,Prabowo akan membentuk Komite Pemantau dan Aku...,Prabowo akan membentuk Komite Pemantau dan Aku...
...,...,...
41,Pakar menilai para pejabat BPI Danantara punya...,Pakar menilai para pejabat BPI Danantara punya...
55,The Backstreet Boys of Danantara Econ For Intr...,The Backstreet Boys of Danantara Econ For Intr...
67,"Oleh: Dewa Putu Adi Wibawa, S.H.\n\nPresiden P...","Oleh: Dewa Putu Adi Wibawa , S.H.\n\nPresiden ..."
66,"JIKA tidak ada aral melintang, Senin (24/2/202...","JIKA tidak ada aral melintang , Senin ( 24/2/2..."


In [48]:
df_negatif_aug['content'] = df_negatif_aug['augmented_content']
df_negatif_aug = df_negatif_aug.drop(columns=['augmented_content'])

# df netral diaugmentasikan 51 berita

In [49]:
df_netral = df_star[df_star['label']=='netral']

In [50]:
df_netral_aug = df_netral.sample(n=51, random_state=42, replace=True)

In [51]:
df_netral_aug['augmented_content'] = df_netral_aug['content'].apply(lambda x: augment_text_with_custom_synonyms(
    x, nlp, indonesian_synonyms_dict, synonym_rate=0.6, random_seed=42
))

In [52]:
df_netral_aug[['content', 'augmented_content']]

,content,augmented_content
156,Danantara: Bayang-Bayang Pro dan Kontra Hendy ...,Danantara: Bayang-Bayang Pro dan Kontra Hendy ...
112,Menteri ATR/Kepala BPN Nusron Wahid mengatakan...,Menteri ATR/ Kepala BPN Nusron Wahid mengataka...
61,Danantara A New Era of State Asset Management ...,Danantara A New Era of State Asset Management ...
170,"Jakarta, CNBC Indonesia - Menteri Sekretaris N...","Jakarta , CNBC Indonesia - Menteri Sekretaris ..."
33,Dalam rangka optimalisasi kekayaan sekaligus a...,Dalam rangka optimalisasi kekayaan sekaligus a...
76,Jakarta – Kementerian Keuangan (Kemenkeu) menc...,Jakarta – Kementerian Keuangan (Kemenkeu) menc...
156,Danantara: Bayang-Bayang Pro dan Kontra Hendy ...,Danantara: Bayang-Bayang Pro dan Kontra Hendy ...
72,ILUSTRASI. Gedung kantor Badan Pengelola Inves...,ILUSTRASI . Gedung kantor Badan Pengelola Inve...
84,Ketua Dewan Pengawas (Dewas) BPI Danantara yan...,Ketua Dewan Pengawas (Dewas) BPI Danantara yan...
47,KOMPAS.com - Media sosial diramaikan dengan se...,KOMPAS.com - Media sosial diramaikan dengan se...


In [53]:
df_netral_aug['content'] = df_netral_aug['augmented_content']
df_netral_aug = df_netral_aug.drop(columns=['augmented_content'])

In [54]:
df_balanced = pd.concat([df_positif, df_negatif, df_negatif_aug, df_netral, df_netral_aug])

In [55]:
df_balanced.drop(columns=['Unnamed: 15', 'Unnamed: 16'], inplace=True)

In [56]:
df_balanced.info()

<class 'pandas.core.frame.DataFrame'>
Index: 300 entries, 160 to 75
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   title           300 non-null    object        
 1   authors         254 non-null    object        
 2   source          300 non-null    object        
 3   published_date  300 non-null    datetime64[ns]
 4   summary         297 non-null    object        
 5   content         300 non-null    object        
 6   url             300 non-null    object        
 7   tag             297 non-null    object        
 8   polarity        300 non-null    float64       
 9   subjectivity    300 non-null    float64       
 10  original        300 non-null    object        
 11  tokens_awal     300 non-null    object        
 12  no_stopwords    300 non-null    object        
 13  stemmed         300 non-null    object        
 14  label           300 non-null    object        
dtypes: datetim

In [57]:
# prompt: print distribusinya

print(df_balanced['label'].value_counts())

label
positif    100
Negatif    100
netral     100
Name: count, dtype: int64


In [58]:
df_balanced = df_balanced.drop(columns=[ 'subjectivity', 'original', 'tokens_awal',  'no_stopwords', 'stemmed'])
df_balanced.info()

<class 'pandas.core.frame.DataFrame'>
Index: 300 entries, 160 to 75
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   title           300 non-null    object        
 1   authors         254 non-null    object        
 2   source          300 non-null    object        
 3   published_date  300 non-null    datetime64[ns]
 4   summary         297 non-null    object        
 5   content         300 non-null    object        
 6   url             300 non-null    object        
 7   tag             297 non-null    object        
 8   polarity        300 non-null    float64       
 9   label           300 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(8)
memory usage: 25.8+ KB


In [59]:
!pip install Sastrawi

In [60]:
import re
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Load stopwords dan stemmer Bahasa Indonesia
factory_stop = StopWordRemoverFactory()
stopwords = factory_stop.get_stop_words()

factory_stem = StemmerFactory()
stemmer = factory_stem.create_stemmer()

# Fungsi preprocessing manual
def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', ' ', text)   # hapus simbol dan angka
    tokens = text.split()
    tokens = [t for t in tokens if t not in stopwords]  # stopword removal
    tokens = [stemmer.stem(t) for t in tokens]          # stemming
    return ' '.join(tokens)


In [61]:
# prompt: apply preprocess ke df content

df_balanced['content_clean'] = df_balanced['content'].apply(preprocess)
df_balanced

,title,authors,source,published_date,summary,content,url,tag,polarity,label,content_clean
160,Danantara: Membangun Masa Depan Investasi Indo...,"Ardian Fikri Rizki, S.M.",ardianfikri.medium.com,2024-11-06 05:06:03,"Peluncuran lembaga baru, Daya Anagata Nusantar...","Pendahuluan\n\nPeluncuran lembaga baru, Daya A...",https://ardianfikri.medium.com/danantara-memba...,local_news,0.117424,positif,dahulu luncur lembaga baru daya anagata nusant...
181,Danantara Raup Dividen Rp 49 Triliun dari BMRI...,"Romys Binekasri, Https, Www.Facebook.Com Cnbci...",cnbcindonesia.com,2025-04-24 18:10:16,BPI Danantara mendapat kucuran dividen dari du...,"Jakarta, CNBC Indonesia — Badan Pengelola Inve...",https://www.cnbcindonesia.com/market/202504241...,economy,0.064881,positif,jakarta cnbc indonesia badan kelola investasi ...
37,Luhut Klaim Danantara Dikelola Orang Profesion...,"Anisa Indraini, Https, Www.Facebook.Com Detikf...",finance.detik.com,2025-02-20 14:31:28,Ketua DEN Luhut Binsar Pandjaitan mengklaim Ba...,Ketua Dewan Ekonomi Nasional (DEN) Luhut Binsa...,https://finance.detik.com/berita-ekonomi-bisni...,economy,0.077388,positif,ketua dewan ekonomi nasional den luhut binsar ...
69,Pemerintah Diminta Bangun Strategi Komunikasi ...,Oleh,ugm.ac.id,2025-03-05 02:32:59,Danantara resmi diluncurkan pada 24 Februari l...,Danantara resmi diluncurkan pada 24 Februari l...,https://ugm.ac.id/id/berita/pemerintah-diminta...,opinion,0.202315,positif,danantara resmi luncur februari lalu jadi binc...
122,CIO Danantara Pandu Sjahrir Bertemu Menkeu AS ...,"Https, Www.Facebook.Com Cnnindonesia",cnnindonesia.com,2025-05-06 20:30:12,CIO BPI Danantara Pandu Sjahrir bertemu dengan...,--\n\nChief of Investment Officer (CIO) Badan ...,https://www.cnnindonesia.com/ekonomi/202505061...,economy,0.051339,positif,chief of investment officer cio badan kelola i...
...,...,...,...,...,...,...,...,...,...,...,...
60,Dasco Soal Erick Thohir Jadi Dewas Danantara,"Jessica Gabriela Soehandoko, Penulis, Editor",kabar24.bisnis.com,2025-02-04 15:00:00,Dasco merespons isu Menteri BUMN Erick Thohir ...,Dasco merespons isu Menteri BUMN Erick Thohir ...,https://kabar24.bisnis.com/read/20250204/15/18...,local_news,0.043478,netral,dasco respons isu menteri bumn erick thohir ja...
36,"Danantara Resmi Dibentuk, Begini Pandangan dar...","Reporter, Editor",tvonenews.com,2025-02-24 09:21:49,Tidak dihantui oleh ketentuan-ketentuan UU.,Tidak dihantui oleh ketentuan-ketentuan UU . J...,https://www.tvonenews.com/berita/303746-danant...,local_news,0.230000,netral,hantu tentu tentu uu jakarta tvonenews com sen...
97,Pakar Ekonomi UNAIR Beri Tanggapan Terkait Pel...,NaN,unair.ac.id,2025-03-18 01:45:36,Presiden Republik Indonesia Prabowo Subianto r...,UNAIR NEWS – Presiden Republik Indonesia Prabo...,https://unair.ac.id/pakar-ekonomi-unair-beri-t...,local_news,0.091414,netral,unair news presiden republik indonesia prabowo...
2,Pro dan Kontra Danantara,NaN,medium.com,2025-02-22 04:01:59,"Danantara, atau secara resmi dikenal sebagai B...",Pro dan Kontra Danantara YONGOPI Follow 3 min ...,https://medium.com/@yongopi/pro-dan-kontra-dan...,economy,0.130567,netral,pro kontra danantara yongopi follow min read f...


In [63]:
df_balanced.to_excel('/content/drive/My Drive/PBA/shofie_only/data_balanced_100.xlsx', index=False)